### File to evaluate difference in utilising isLRG directly and the full DESI pipeline

##### Existing Pipeline

In [2]:
from astropy.io import fits
import os
import wget
import numpy as np
import time
import pandas as pd
from brick import Brick
import telegram_send


In [51]:

""" File to download, process, classify and delete galaxies from DR9 all in one"""

""" Defining area to download, how many bricks to download in one session and which storage to use (Astrodisk is the name of a hardrive)"""
area = 'south'
device = 'Astrostick'
bricks_to_classify = 30000
south_survey_is_south =  True

## ToDo: Create special folder in astrodisk with dedicated bricks
hdulistBricksSouthSummary = fits.open('../../bricks_data/survey-bricks-dr9-south.fits')
data_south = hdulistBricksSouthSummary[1].data
brickname_south = data_south.field('brickname')
brickid_south = data_south.field('brickid')
south_survey_is_south = data_south.field('survey_primary')

In [55]:
# Everything is times in order to measure when the pipeline slows down
start = time.time()

print()
print(f"=============================== Process {area} ..... ==================================")
print()

bricks_name = []
bricks_path = []

# Getting already downloaded files from the Harddrive:


for filename in os.listdir(f'/Volumes/{device}/bricks_data/{area}/'):
    brickn = filename.replace("tractor-", "")
    brickn = brickn.replace(".fits", "")
    bricks_path.append(filename)
    bricks_name.append(brickn)

bricks_name.pop()
bricks_path.pop()
"""for filename in os.listdir(f'../../bricks_data/tractor/'):
    if '.fits' not in filename:
        continue
    brickn = filename.replace("tractor-", "")
    brickn = brickn.replace(".fits", "")
    bricks_path.append(filename)
    bricks_name.append(brickn)"""


# Define empty Dataframes that will hold the information on stars and galaxies
df_galaxy = pd.DataFrame(columns=['BrickID', 'RA', 'DEC', 'LRG', 'ELG', 'QSO'])
df_stars = pd.DataFrame(columns=['RA', 'DEC', 'GMAG', 'RMAG', 'ZMAG'])



# Prints information on the  current session e.g. how many bricks are left --> all the code until here takes a few minutes to complete
print(f"No of bricks to classify in {area}: {len(bricks_name)} ")
print("Time taken for bricks left extraction: ", round(((time.time() - start) / 60), 2))


# There have been problems with very few bricks that were not found on the servers, this code is only to avoid the script from crashing here
c = 0
problem_bricks = []
inter = time.time()


# This is the actual loop doing the classification for the bricks that are missing from the catalogue:

for i, brickname in enumerate(bricks_name):

    # Download Brick



    brickid = brickid_south[np.where(brickname_south == brickname)]

    # North Bricks
    # brickid = brickid_north[np.where(brickname_north == brickname)]

    if len(brickid > 0):
        brickid = brickid[0]
    else:
        brickid = 0

    # Open Brick



    #hdu = fits.open(f'../../bricks_data/tractor/tractor-{brickname}.fits')
    hdu = fits.open(f'/Volumes/{device}/bricks_data/{area}/tractor-{brickname}.fits')
    data = hdu[1].data

    # Define the Brick Object  --> in brick.py
    brick = Brick(data)

    # south = north_survey_is_south[np.where(brickid_north == brickid)]

    south = south_survey_is_south[np.where(brickid_south == brickid)]
    if len(south) > 0:
        south = south[0]
    else:
        south = True

    ## Enable this is classifying North Objects
    # south = north_survey_is_south[np.where(brickid_north == brickid)][0]

    # Initialise Brick Object
    brick.initialise_brick_for_galaxy_classification(south)

    # Classify Brick objects into categories --> takes under 1 second after optimisation
    target_objects = brick.classify_galaxies()

    # Appending one empty line per brick to be sure that all bricks are extracted
    df_galaxy = df_galaxy.append({'BrickID': brickid, 'RA': np.nan, 'DEC': np.nan, 'LRG': 0, 'ELG': 0, 'QSO': 0},
                                 ignore_index=True)

    support_df = pd.DataFrame(target_objects,
                              columns=['BrickID', 'RA', 'DEC', 'LRG', 'ELG', 'QSO'])

    df_galaxy = df_galaxy.append(support_df)

    # Repeat steps for stellar objects

    brick.initialise_brick_for_stellar_density()

    #stars = brick.get_stellar_objects()

    #support_df = pd.DataFrame(stars, columns=['RA', 'DEC', 'GMAG', 'RMAG', 'ZMAG'])
    #df_stars = df_stars.append(support_df)

    # Every 100 objects, the newly classified objects are added to the existing catalogue to avoid massive reruns when the script crashes

    """if i % 100 == 0:
        print()
        print(i / (bricks_to_classify / 100), '%')
        df_galaxy = df_galaxy.astype(
            {'BrickID': 'int32', 'LRG': 'int8', 'ELG': 'int8', 'QSO': 'int8'})
        df_galaxy.to_csv(f'../../bricks_data/galaxy_catalogue_{area}.csv', mode='a', index=False, header=False)
        df_stars.to_csv(f'../../bricks_data/stellar_catalogue_{area}.csv', mode='a', index=False, header=False)
        # df_galaxy.to_csv('../../bricks_data/galaxy_catalogue_sample_profiling.csv', index=False, header=False)
        # df_stars.to_csv('../../bricks_data/stellar_catalogue_sample_profiling.csv', index=False, header=False)
        df_galaxy = df_galaxy[0:0]
        df_stars = df_stars[0:0]"""

    # This script used to send me updates to my phone using a Telegram Bot, so i knew when it crashed or it was completed

    # Remove Downloaded Brick
    # os.remove(f'/Volumes/{device}/bricks_data/{area}/tractor-{brickname}.fits')

    # Stop the loop when the defined number of bricks was classified, if this number is greater than remaining bricks, script will run till all bricks are finished
    if i > bricks_to_classify:
        break

    if i % 100 == 0:
        print(f" Brick {area} processed: ", brickname, ", Brick ", i, " of ", bricks_to_classify)

df_galaxy = df_galaxy.astype(
    {'BrickID': 'int32', 'LRG': 'int8', 'ELG': 'int8', 'QSO': 'int8'})
#df_galaxy.to_csv(f'../../bricks_data/galaxy_catalogue_{area}.csv', mode='a', index=False, header=False)
#df_stars.to_csv(f'../../bricks_data/stellar_catalogue_{area}.csv', mode='a', index=False, header=False)
#df_galaxy = df_galaxy[0:0]
#df_stars = df_stars[0:0]
print()
print(f"=============================== Download {area} completed ==================================")
print()


# Prints session statistics upon completion
print("Minutes taken for: ", i, " bricks: ", round(((time.time() - start) / 60), 2))
print("Hours taken for: ", i, " bricks: ", round(((time.time() - start) / 3600), 2))
# message = f'++++++ Finished {bricks_to_classify} bricks. Avg. Bandwidths: {round(((time.time() - start) / bricks_to_classify), 2)} seconds per brick ++++++'




=============================== Process south ..... ==================================

No of bricks to classify in south: 1051 
Time taken for bricks left extraction:  0.0
 Brick south processed:  2443p257 , Brick  0  of  30000
 Brick south processed:  0531m550 , Brick  100  of  30000
 Brick south processed:  2133m022 , Brick  200  of  30000
 Brick south processed:  0813m195 , Brick  300  of  30000
 Brick south processed:  0254p060 , Brick  400  of  30000
 Brick south processed:  3142p100 , Brick  500  of  30000
 Brick south processed:  0171p265 , Brick  600  of  30000
 Brick south processed:  1295p062 , Brick  700  of  30000
 Brick south processed:  0405p090 , Brick  800  of  30000
 Brick south processed:  0346m290 , Brick  900  of  30000
 Brick south processed:  3500p065 , Brick  1000  of  30000

=============================== Download south completed ==================================

Minutes taken for:  1050  bricks:  15.77
Hours taken for:  1050  bricks:  0.26


0046m020


In [57]:

df_galaxy.dropna(inplace=True)


LRG = df_galaxy[df_galaxy.LRG == 1]
print("LRG:", len(LRG))

ELG = df_galaxy[df_galaxy.ELG == 1]
print("ELG:", len(ELG))

QSO = df_galaxy[df_galaxy.QSO == 1]
print("QSO:", len(QSO))


print(len(df_galaxy))
print(len(LRG) + len(ELG) + len(QSO))

print(len(df_galaxy) - (len(LRG) + len(ELG) + len(QSO)))

LRG: 41081
ELG: 154533
QSO: 195119
372459
390733
-18274


### Utilising DesiHub Pipeline

In [63]:
from desitarget.cuts import select_targets
filenames = []

for filename in os.listdir(f'/Volumes/{device}/bricks_data/{area}/'):
    if '.fits' not in filename:
        continue
    filenames.append(f'/Volumes/{device}/bricks_data/{area}/{filename}')

filenames.pop()
print(len(filenames))

1051


In [59]:
start = time.time()

lrg = select_targets(
    infiles=filenames,numproc=1,qso_selection='colorcuts', nside=None, gaiasub=False,
    tcnames= ['LRG'], backup=False
)

print("Minutes taken for: ", i, " bricks: ", round(((time.time() - start) / 60), 2))
print("Hours taken for: ", i, " bricks: ", round(((time.time() - start) / 3600), 2))



INFO:cuts.py:2942:select_targets: Running on the main survey
INFO:cuts.py:3059:_update_status: 20/1051 files; 0.9 secs/file; 0.3 total mins elapsed
INFO:cuts.py:3059:_update_status: 40/1051 files; 0.9 secs/file; 0.6 total mins elapsed
INFO:cuts.py:3059:_update_status: 60/1051 files; 0.9 secs/file; 0.9 total mins elapsed
INFO:cuts.py:3059:_update_status: 80/1051 files; 0.9 secs/file; 1.2 total mins elapsed
INFO:cuts.py:3059:_update_status: 100/1051 files; 0.9 secs/file; 1.5 total mins elapsed
INFO:cuts.py:3059:_update_status: 120/1051 files; 0.9 secs/file; 1.8 total mins elapsed
INFO:cuts.py:3059:_update_status: 140/1051 files; 0.9 secs/file; 2.1 total mins elapsed
INFO:cuts.py:3059:_update_status: 160/1051 files; 0.9 secs/file; 2.4 total mins elapsed
INFO:cuts.py:3059:_update_status: 180/1051 files; 0.9 secs/file; 2.7 total mins elapsed
INFO:cuts.py:3059:_update_status: 200/1051 files; 0.9 secs/file; 3.0 total mins elapsed
INFO:cuts.py:3059:_update_status: 220/1051 files; 0.9 secs/file

KeyboardInterrupt: 

In [ ]:
start = time.time()


elg = select_targets(
    infiles=filenames,numproc=1,qso_selection='colorcuts', nside=None, gaiasub=False,
    tcnames= ['ELG'], backup=False
)

print("Minutes taken for: ", i, " bricks: ", round(((time.time() - start) / 60), 2))
print("Hours taken for: ", i, " bricks: ", round(((time.time() - start) / 3600), 2))



In [ ]:
start = time.time()


qso = select_targets(
    infiles=filenames,numproc=1,qso_selection='colorcuts', nside=None, gaiasub=False,
    tcnames= ['QSO'], backup=False)


print("Minutes taken for: ", i, " bricks: ", round(((time.time() - start) / 60), 2))
print("Hours taken for: ", i, " bricks: ", round(((time.time() - start) / 3600), 2))



INFO:cuts.py:2942:select_targets: Running on the main survey
INFO:cuts.py:3059:_update_status: 20/1051 files; 0.9 secs/file; 0.3 total mins elapsed
INFO:cuts.py:3059:_update_status: 40/1051 files; 0.9 secs/file; 0.6 total mins elapsed
INFO:cuts.py:3059:_update_status: 60/1051 files; 0.9 secs/file; 0.9 total mins elapsed
INFO:cuts.py:3059:_update_status: 80/1051 files; 0.9 secs/file; 1.2 total mins elapsed
INFO:cuts.py:3059:_update_status: 100/1051 files; 0.9 secs/file; 1.5 total mins elapsed


In [ ]:
start = time.time()

qso_rf = select_targets(
    infiles=filenames,numproc=1,qso_selection='randomforest', nside=None, gaiasub=False,
    tcnames= ['QSO'], backup=False)


print("Minutes taken for: ", i, " bricks: ", round(((time.time() - start) / 60), 2))
print("Hours taken for: ", i, " bricks: ", round(((time.time() - start) / 3600), 2))


In [ ]:
start = time.time()

res = select_targets(
    infiles=filenames,numproc=1,qso_selection='colorcuts', nside=None, gaiasub=False,
    tcnames= ['LRG', 'ELG', 'QSO'], backup=False)


print("Minutes taken for: ", i, " bricks: ", round(((time.time() - start) / 60), 2))
print("Hours taken for: ", i, " bricks: ", round(((time.time() - start) / 3600), 2))


In [ ]:
print("LRG:", len(lrg))
print("ELG:", len(elg))

In [ ]:
print(len(lrg) + len(elg) + len(qso))

print("LRG:", len(lrg))
print("ELG:", len(elg))
print("QSO:", len(qso))
print("QSO_RF:", len(qso_rf))

In [ ]:


filenames = []

for filename in os.listdir(f'../../bricks_data/tractor/'):
    if '.fits' not in filename:
        continue
    filenames.append(f'/Volumes/{device}/bricks_data/{area}/{filename}')